In [13]:
from tensorflow.keras.models import load_model

model = load_model('hand_gestures.h5')

In [14]:
import cv2 as cv
import numpy as np
import mediapipe as mp

capture = cv.VideoCapture(0)
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

switch = {
    0: "Closed Hand",
    1: "Crossed Fingers",
    2: "Five / Open Hand",
    3: "Four",
    4: "Okay Sign",
    5: "One / Point Sideways",
    6: "Open Hand",
    7: "Two / Peace Sign",
    8: "Point at Camera",
    9: "Three",
    10: "Thumbs Down",
    11: "Thumbs Up"
}

def preprocess_frame(frame):
    frame = cv.resize(frame, (224, 224))
    frame = frame / 255.0
    frame = np.reshape(frame, (1, 224, 224, 3))
    return frame

while True:
    ret, frame = capture.read()
    if not ret:
        break

    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            h, w, c = frame.shape
            x_min = int(min([lm.x for lm in hand_landmarks.landmark]) * w + 100)
            x_max = int(max([lm.x for lm in hand_landmarks.landmark]) * w + 100)
            y_min = int(min([lm.y for lm in hand_landmarks.landmark]) * h + 100)
            y_max = int(max([lm.y for lm in hand_landmarks.landmark]) * h + 100)

            hand_roi = frame[y_min:y_max, x_min:x_max]

            if hand_roi.size != 0:
                preprocessed_frame = preprocess_frame(hand_roi)

                prediction = model.predict(preprocessed_frame)
                gesture = switch.get(np.argmax(prediction))

                cv.putText(frame, f'Gesture: {gesture}', (x_min, y_min - 10), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv.imshow('Live Gesture Recognition', frame)

    # Exit if 'q' is pressed
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

capture.release()
cv.destroyAllWindows()

1/1 [==============================] - 0s 33ms/step
